## CONVAE

In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-10-16 13:11:15.550334: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-16 13:11:15.550373: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [4]:
SKAB_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter3/train/'
SKAB_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter3/test/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']


In [9]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/ConvAE_HYP.csv')


input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
source_dict_results_test =  {}
source_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
    model  = ConvAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
    test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    source_dict_results_test[file] = test_df
    source_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_ConvAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_ConvAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])

model = ConvAEModel(layer1=40, layer2=20, encoding_dim=6)
model.fit(train_merge_df, train_merge_df, epochs=100, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_test =  {}
mtl_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_ConvAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_ConvAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

Epoch 1/100
70/70 - 1s - loss: 0.0584 - val_loss: 0.0256 - 858ms/epoch - 12ms/step
Epoch 2/100
70/70 - 0s - loss: 0.0248 - val_loss: 0.0193 - 213ms/epoch - 3ms/step
Epoch 3/100
70/70 - 0s - loss: 0.0172 - val_loss: 0.0099 - 216ms/epoch - 3ms/step
Epoch 4/100
70/70 - 0s - loss: 0.0093 - val_loss: 0.0094 - 206ms/epoch - 3ms/step
Epoch 5/100
70/70 - 0s - loss: 0.0071 - val_loss: 0.0086 - 232ms/epoch - 3ms/step
Epoch 6/100
70/70 - 0s - loss: 0.0060 - val_loss: 0.0080 - 211ms/epoch - 3ms/step
Epoch 7/100
70/70 - 0s - loss: 0.0057 - val_loss: 0.0077 - 220ms/epoch - 3ms/step
Epoch 8/100
70/70 - 0s - loss: 0.0055 - val_loss: 0.0077 - 229ms/epoch - 3ms/step
Epoch 9/100
70/70 - 0s - loss: 0.0054 - val_loss: 0.0073 - 250ms/epoch - 4ms/step
Epoch 10/100
70/70 - 0s - loss: 0.0054 - val_loss: 0.0071 - 264ms/epoch - 4ms/step
Epoch 11/100
70/70 - 0s - loss: 0.0053 - val_loss: 0.0073 - 255ms/epoch - 4ms/step
Epoch 12/100
70/70 - 0s - loss: 0.0053 - val_loss: 0.0071 - 220ms/epoch - 3ms/step
Epoch 13/100

Epoch 100/100
70/70 - 0s - loss: 0.0038 - val_loss: 0.0053 - 204ms/epoch - 3ms/step
Epoch 1/100
35/35 - 1s - loss: 0.0724 - val_loss: 0.0635 - 1s/epoch - 30ms/step
Epoch 2/100
35/35 - 0s - loss: 0.0377 - val_loss: 0.0339 - 146ms/epoch - 4ms/step
Epoch 3/100
35/35 - 0s - loss: 0.0291 - val_loss: 0.0294 - 138ms/epoch - 4ms/step
Epoch 4/100
35/35 - 0s - loss: 0.0264 - val_loss: 0.0223 - 149ms/epoch - 4ms/step
Epoch 5/100
35/35 - 0s - loss: 0.0244 - val_loss: 0.0178 - 148ms/epoch - 4ms/step
Epoch 6/100
35/35 - 0s - loss: 0.0228 - val_loss: 0.0160 - 154ms/epoch - 4ms/step
Epoch 7/100
35/35 - 0s - loss: 0.0211 - val_loss: 0.0145 - 140ms/epoch - 4ms/step
Epoch 8/100
35/35 - 0s - loss: 0.0194 - val_loss: 0.0140 - 146ms/epoch - 4ms/step
Epoch 9/100
35/35 - 0s - loss: 0.0174 - val_loss: 0.0129 - 152ms/epoch - 4ms/step
Epoch 10/100
35/35 - 0s - loss: 0.0151 - val_loss: 0.0123 - 147ms/epoch - 4ms/step
Epoch 11/100
35/35 - 0s - loss: 0.0131 - val_loss: 0.0106 - 139ms/epoch - 4ms/step
Epoch 12/100
3

Epoch 99/100
35/35 - 0s - loss: 0.0060 - val_loss: 0.0078 - 152ms/epoch - 4ms/step
Epoch 100/100
35/35 - 0s - loss: 0.0059 - val_loss: 0.0077 - 154ms/epoch - 4ms/step
Epoch 1/100
70/70 - 1s - loss: 0.0679 - val_loss: 0.0438 - 1s/epoch - 19ms/step
Epoch 2/100
70/70 - 0s - loss: 0.0271 - val_loss: 0.0245 - 186ms/epoch - 3ms/step
Epoch 3/100
70/70 - 0s - loss: 0.0183 - val_loss: 0.0109 - 194ms/epoch - 3ms/step
Epoch 4/100
70/70 - 0s - loss: 0.0081 - val_loss: 0.0088 - 181ms/epoch - 3ms/step
Epoch 5/100
70/70 - 0s - loss: 0.0066 - val_loss: 0.0083 - 182ms/epoch - 3ms/step
Epoch 6/100
70/70 - 0s - loss: 0.0061 - val_loss: 0.0076 - 178ms/epoch - 3ms/step
Epoch 7/100
70/70 - 0s - loss: 0.0059 - val_loss: 0.0081 - 219ms/epoch - 3ms/step
Epoch 8/100
70/70 - 0s - loss: 0.0058 - val_loss: 0.0079 - 192ms/epoch - 3ms/step
Epoch 9/100
70/70 - 0s - loss: 0.0057 - val_loss: 0.0076 - 187ms/epoch - 3ms/step
Epoch 10/100
70/70 - 0s - loss: 0.0056 - val_loss: 0.0074 - 183ms/epoch - 3ms/step
Epoch 11/100
7

Epoch 98/100
70/70 - 0s - loss: 0.0040 - val_loss: 0.0056 - 177ms/epoch - 3ms/step
Epoch 99/100
70/70 - 0s - loss: 0.0040 - val_loss: 0.0056 - 180ms/epoch - 3ms/step
Epoch 100/100
70/70 - 0s - loss: 0.0040 - val_loss: 0.0056 - 186ms/epoch - 3ms/step
Epoch 1/200
35/35 - 1s - loss: 0.0777 - val_loss: 0.0845 - 1s/epoch - 31ms/step
Epoch 2/200
35/35 - 0s - loss: 0.0437 - val_loss: 0.0296 - 143ms/epoch - 4ms/step
Epoch 3/200
35/35 - 0s - loss: 0.0258 - val_loss: 0.0201 - 153ms/epoch - 4ms/step
Epoch 4/200
35/35 - 0s - loss: 0.0159 - val_loss: 0.0105 - 141ms/epoch - 4ms/step
Epoch 5/200
35/35 - 0s - loss: 0.0105 - val_loss: 0.0085 - 153ms/epoch - 4ms/step
Epoch 6/200
35/35 - 0s - loss: 0.0087 - val_loss: 0.0082 - 144ms/epoch - 4ms/step
Epoch 7/200
35/35 - 0s - loss: 0.0073 - val_loss: 0.0078 - 145ms/epoch - 4ms/step
Epoch 8/200
35/35 - 0s - loss: 0.0061 - val_loss: 0.0075 - 145ms/epoch - 4ms/step
Epoch 9/200
35/35 - 0s - loss: 0.0055 - val_loss: 0.0068 - 141ms/epoch - 4ms/step
Epoch 10/200
3

Epoch 97/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0029 - 142ms/epoch - 4ms/step
Epoch 98/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0030 - 142ms/epoch - 4ms/step
Epoch 99/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0030 - 141ms/epoch - 4ms/step
Epoch 100/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0029 - 149ms/epoch - 4ms/step
Epoch 101/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0028 - 149ms/epoch - 4ms/step
Epoch 102/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0029 - 142ms/epoch - 4ms/step
Epoch 103/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0028 - 140ms/epoch - 4ms/step
Epoch 104/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0028 - 141ms/epoch - 4ms/step
Epoch 105/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0027 - 158ms/epoch - 5ms/step
Epoch 106/200
35/35 - 0s - loss: 0.0016 - val_loss: 0.0028 - 148ms/epoch - 4ms/step
Epoch 107/200
35/35 - 0s - loss: 0.0015 - val_loss: 0.0027 - 141ms/epoch - 4ms/step
Epoch 108/200
35/35 - 0s - loss: 0.0015 - val_loss: 0.0026 - 143ms/epoch - 4ms/

...

Epoch 76/100
35/35 - 0s - loss: 0.0014 - val_loss: 0.0024 - 146ms/epoch - 4ms/step
Epoch 77/100
35/35 - 0s - loss: 0.0014 - val_loss: 0.0027 - 137ms/epoch - 4ms/step
Epoch 78/100
35/35 - 0s - loss: 0.0014 - val_loss: 0.0025 - 139ms/epoch - 4ms/step
Epoch 79/100
35/35 - 0s - loss: 0.0014 - val_loss: 0.0024 - 137ms/epoch - 4ms/step
Epoch 80/100
35/35 - 0s - loss: 0.0014 - val_loss: 0.0022 - 140ms/epoch - 4ms/step
Epoch 81/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0022 - 164ms/epoch - 5ms/step
Epoch 82/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0021 - 144ms/epoch - 4ms/step
Epoch 83/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0023 - 143ms/epoch - 4ms/step
Epoch 84/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0023 - 145ms/epoch - 4ms/step
Epoch 85/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0021 - 155ms/epoch - 4ms/step
Epoch 86/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0022 - 155ms/epoch - 4ms/step
Epoch 87/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0023 - 158ms/epoch - 5ms/step
Epoc

Epoch 75/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0038 - 146ms/epoch - 4ms/step
Epoch 76/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0041 - 147ms/epoch - 4ms/step
Epoch 77/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0041 - 132ms/epoch - 4ms/step
Epoch 78/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0044 - 144ms/epoch - 4ms/step
Epoch 79/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0036 - 136ms/epoch - 4ms/step
Epoch 80/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0037 - 143ms/epoch - 4ms/step
Epoch 81/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0039 - 144ms/epoch - 4ms/step
Epoch 82/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0039 - 152ms/epoch - 4ms/step
Epoch 83/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0040 - 138ms/epoch - 4ms/step
Epoch 84/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0038 - 157ms/epoch - 4ms/step
Epoch 85/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0037 - 156ms/epoch - 4ms/step
Epoch 86/100
35/35 - 0s - loss: 0.0034 - val_loss: 0.0038 - 147ms/epoch - 4ms/step
Epoc

Epoch 74/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0057 - 210ms/epoch - 3ms/step
Epoch 75/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0059 - 209ms/epoch - 3ms/step
Epoch 76/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0059 - 201ms/epoch - 3ms/step
Epoch 77/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0057 - 221ms/epoch - 3ms/step
Epoch 78/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0056 - 217ms/epoch - 3ms/step
Epoch 79/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0054 - 222ms/epoch - 3ms/step
Epoch 80/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0055 - 208ms/epoch - 3ms/step
Epoch 81/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0056 - 212ms/epoch - 3ms/step
Epoch 82/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0056 - 208ms/epoch - 3ms/step
Epoch 83/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0052 - 215ms/epoch - 3ms/step
Epoch 84/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0054 - 200ms/epoch - 3ms/step
Epoch 85/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0053 - 211ms/epoch - 3ms/step
Epoc

Epoch 73/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0053 - 132ms/epoch - 4ms/step
Epoch 74/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0053 - 136ms/epoch - 4ms/step
Epoch 75/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0055 - 168ms/epoch - 5ms/step
Epoch 76/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0054 - 158ms/epoch - 5ms/step
Epoch 77/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0054 - 145ms/epoch - 4ms/step
Epoch 78/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0051 - 140ms/epoch - 4ms/step
Epoch 79/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0052 - 135ms/epoch - 4ms/step
Epoch 80/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0052 - 136ms/epoch - 4ms/step
Epoch 81/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0051 - 147ms/epoch - 4ms/step
Epoch 82/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0055 - 173ms/epoch - 5ms/step
Epoch 83/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0052 - 182ms/epoch - 5ms/step
Epoch 84/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0053 - 176ms/epoch - 5ms/step
Epoc

Epoch 71/100
109/109 - 0s - loss: 0.0013 - val_loss: 0.0060 - 307ms/epoch - 3ms/step
Epoch 72/100
109/109 - 0s - loss: 0.0013 - val_loss: 0.0060 - 288ms/epoch - 3ms/step
Epoch 73/100
109/109 - 0s - loss: 0.0013 - val_loss: 0.0060 - 287ms/epoch - 3ms/step
Epoch 74/100
109/109 - 0s - loss: 0.0013 - val_loss: 0.0059 - 274ms/epoch - 3ms/step
Epoch 75/100
109/109 - 0s - loss: 0.0013 - val_loss: 0.0060 - 287ms/epoch - 3ms/step
Epoch 76/100
109/109 - 0s - loss: 0.0013 - val_loss: 0.0059 - 282ms/epoch - 3ms/step
Epoch 77/100
109/109 - 0s - loss: 0.0012 - val_loss: 0.0059 - 289ms/epoch - 3ms/step
Epoch 78/100
109/109 - 0s - loss: 0.0012 - val_loss: 0.0058 - 300ms/epoch - 3ms/step
Epoch 79/100
109/109 - 0s - loss: 0.0012 - val_loss: 0.0058 - 279ms/epoch - 3ms/step
Epoch 80/100
109/109 - 0s - loss: 0.0012 - val_loss: 0.0058 - 279ms/epoch - 3ms/step
Epoch 81/100
109/109 - 0s - loss: 0.0012 - val_loss: 0.0058 - 280ms/epoch - 3ms/step
Epoch 82/100
109/109 - 0s - loss: 0.0012 - val_loss: 0.0058 - 288

## CONVATTNAE

In [2]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-09-08 18:44:33.364244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-08 18:44:33.364331: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [7]:
SKAB_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter2/train/'
SKAB_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter2/test/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']


In [8]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/ConvAttnAE_HYP.csv')


input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
source_dict_results_test =  {}
source_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
    model  = ConvAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
    test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    source_dict_results_test[file] = test_df
    source_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_ConvAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_ConvAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])

model = ConvAttnAEModel(layer1=30, layer2=20, encoding_dim=4)
model.fit(train_merge_df, train_merge_df, epochs=100, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_test =  {}
mtl_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_ConvAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_ConvAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

Epoch 1/100
35/35 - 1s - loss: 0.0760 - val_loss: 0.0900 - 1s/epoch - 32ms/step
Epoch 2/100
35/35 - 0s - loss: 0.0465 - val_loss: 0.0281 - 142ms/epoch - 4ms/step
Epoch 3/100
35/35 - 0s - loss: 0.0270 - val_loss: 0.0290 - 139ms/epoch - 4ms/step
Epoch 4/100
35/35 - 0s - loss: 0.0248 - val_loss: 0.0236 - 140ms/epoch - 4ms/step
Epoch 5/100
35/35 - 0s - loss: 0.0218 - val_loss: 0.0156 - 143ms/epoch - 4ms/step
Epoch 6/100
35/35 - 0s - loss: 0.0188 - val_loss: 0.0146 - 134ms/epoch - 4ms/step
Epoch 7/100
35/35 - 0s - loss: 0.0136 - val_loss: 0.0106 - 137ms/epoch - 4ms/step
Epoch 8/100
35/35 - 0s - loss: 0.0087 - val_loss: 0.0090 - 138ms/epoch - 4ms/step
Epoch 9/100
35/35 - 0s - loss: 0.0077 - val_loss: 0.0089 - 145ms/epoch - 4ms/step
Epoch 10/100
35/35 - 0s - loss: 0.0072 - val_loss: 0.0086 - 139ms/epoch - 4ms/step
Epoch 11/100
35/35 - 0s - loss: 0.0070 - val_loss: 0.0086 - 140ms/epoch - 4ms/step
Epoch 12/100
35/35 - 0s - loss: 0.0068 - val_loss: 0.0085 - 142ms/epoch - 4ms/step
Epoch 13/100
35

Epoch 100/100
35/35 - 0s - loss: 0.0041 - val_loss: 0.0060 - 136ms/epoch - 4ms/step
Epoch 1/100
35/35 - 1s - loss: 0.0782 - val_loss: 0.0895 - 1s/epoch - 33ms/step
Epoch 2/100
35/35 - 0s - loss: 0.0484 - val_loss: 0.0257 - 141ms/epoch - 4ms/step
Epoch 3/100
35/35 - 0s - loss: 0.0286 - val_loss: 0.0265 - 136ms/epoch - 4ms/step
Epoch 4/100
35/35 - 0s - loss: 0.0239 - val_loss: 0.0142 - 139ms/epoch - 4ms/step
Epoch 5/100
35/35 - 0s - loss: 0.0177 - val_loss: 0.0096 - 141ms/epoch - 4ms/step
Epoch 6/100
35/35 - 0s - loss: 0.0105 - val_loss: 0.0088 - 137ms/epoch - 4ms/step
Epoch 7/100
35/35 - 0s - loss: 0.0070 - val_loss: 0.0085 - 140ms/epoch - 4ms/step
Epoch 8/100
35/35 - 0s - loss: 0.0062 - val_loss: 0.0077 - 140ms/epoch - 4ms/step
Epoch 9/100
35/35 - 0s - loss: 0.0059 - val_loss: 0.0070 - 140ms/epoch - 4ms/step
Epoch 10/100
35/35 - 0s - loss: 0.0057 - val_loss: 0.0065 - 133ms/epoch - 4ms/step
Epoch 11/100
35/35 - 0s - loss: 0.0055 - val_loss: 0.0062 - 140ms/epoch - 4ms/step
Epoch 12/100
3

Epoch 99/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0027 - 137ms/epoch - 4ms/step
Epoch 100/100
35/35 - 0s - loss: 0.0013 - val_loss: 0.0026 - 141ms/epoch - 4ms/step
Epoch 1/100
35/35 - 1s - loss: 0.0774 - val_loss: 0.0987 - 1s/epoch - 30ms/step
Epoch 2/100
35/35 - 0s - loss: 0.0607 - val_loss: 0.0488 - 128ms/epoch - 4ms/step
Epoch 3/100
35/35 - 0s - loss: 0.0299 - val_loss: 0.0330 - 127ms/epoch - 4ms/step
Epoch 4/100
35/35 - 0s - loss: 0.0255 - val_loss: 0.0264 - 129ms/epoch - 4ms/step
Epoch 5/100
35/35 - 0s - loss: 0.0227 - val_loss: 0.0154 - 126ms/epoch - 4ms/step
Epoch 6/100
35/35 - 0s - loss: 0.0193 - val_loss: 0.0127 - 132ms/epoch - 4ms/step
Epoch 7/100
35/35 - 0s - loss: 0.0125 - val_loss: 0.0094 - 129ms/epoch - 4ms/step
Epoch 8/100
35/35 - 0s - loss: 0.0079 - val_loss: 0.0087 - 142ms/epoch - 4ms/step
Epoch 9/100
35/35 - 0s - loss: 0.0070 - val_loss: 0.0081 - 128ms/epoch - 4ms/step
Epoch 10/100
35/35 - 0s - loss: 0.0065 - val_loss: 0.0084 - 135ms/epoch - 4ms/step
Epoch 11/100
3

Epoch 98/100
35/35 - 0s - loss: 0.0043 - val_loss: 0.0060 - 131ms/epoch - 4ms/step
Epoch 99/100
35/35 - 0s - loss: 0.0043 - val_loss: 0.0063 - 124ms/epoch - 4ms/step
Epoch 100/100
35/35 - 0s - loss: 0.0043 - val_loss: 0.0059 - 119ms/epoch - 3ms/step
Epoch 1/200
70/70 - 2s - loss: 0.0661 - val_loss: 0.0309 - 2s/epoch - 25ms/step
Epoch 2/200
70/70 - 0s - loss: 0.0276 - val_loss: 0.0250 - 210ms/epoch - 3ms/step
Epoch 3/200
70/70 - 0s - loss: 0.0221 - val_loss: 0.0132 - 213ms/epoch - 3ms/step
Epoch 4/200
70/70 - 0s - loss: 0.0113 - val_loss: 0.0091 - 207ms/epoch - 3ms/step
Epoch 5/200
70/70 - 0s - loss: 0.0072 - val_loss: 0.0085 - 215ms/epoch - 3ms/step
Epoch 6/200
70/70 - 0s - loss: 0.0061 - val_loss: 0.0086 - 222ms/epoch - 3ms/step
Epoch 7/200
70/70 - 0s - loss: 0.0058 - val_loss: 0.0079 - 211ms/epoch - 3ms/step
Epoch 8/200
70/70 - 0s - loss: 0.0057 - val_loss: 0.0081 - 204ms/epoch - 3ms/step
Epoch 9/200
70/70 - 0s - loss: 0.0057 - val_loss: 0.0080 - 206ms/epoch - 3ms/step
Epoch 10/200
7

Epoch 97/200
70/70 - 0s - loss: 0.0037 - val_loss: 0.0049 - 210ms/epoch - 3ms/step
Epoch 98/200
70/70 - 0s - loss: 0.0037 - val_loss: 0.0047 - 194ms/epoch - 3ms/step
Epoch 99/200
70/70 - 0s - loss: 0.0037 - val_loss: 0.0046 - 204ms/epoch - 3ms/step
Epoch 100/200
70/70 - 0s - loss: 0.0037 - val_loss: 0.0046 - 211ms/epoch - 3ms/step
Epoch 101/200
70/70 - 0s - loss: 0.0037 - val_loss: 0.0045 - 201ms/epoch - 3ms/step
Epoch 102/200
70/70 - 0s - loss: 0.0037 - val_loss: 0.0044 - 210ms/epoch - 3ms/step
Epoch 103/200
70/70 - 0s - loss: 0.0036 - val_loss: 0.0044 - 211ms/epoch - 3ms/step
Epoch 104/200
70/70 - 0s - loss: 0.0036 - val_loss: 0.0045 - 204ms/epoch - 3ms/step
Epoch 105/200
70/70 - 0s - loss: 0.0036 - val_loss: 0.0044 - 208ms/epoch - 3ms/step
Epoch 106/200
70/70 - 0s - loss: 0.0036 - val_loss: 0.0043 - 199ms/epoch - 3ms/step
Epoch 107/200
70/70 - 0s - loss: 0.0036 - val_loss: 0.0047 - 206ms/epoch - 3ms/step
Epoch 108/200
70/70 - 0s - loss: 0.0036 - val_loss: 0.0048 - 199ms/epoch - 3ms/

...

35/35 - 0s - loss: 0.0042 - val_loss: 0.0062 - 128ms/epoch - 4ms/step
Epoch 31/100
35/35 - 0s - loss: 0.0042 - val_loss: 0.0058 - 131ms/epoch - 4ms/step
Epoch 32/100
35/35 - 0s - loss: 0.0041 - val_loss: 0.0061 - 129ms/epoch - 4ms/step
Epoch 33/100
35/35 - 0s - loss: 0.0041 - val_loss: 0.0060 - 132ms/epoch - 4ms/step
Epoch 34/100
35/35 - 0s - loss: 0.0041 - val_loss: 0.0058 - 131ms/epoch - 4ms/step
Epoch 35/100
35/35 - 0s - loss: 0.0041 - val_loss: 0.0056 - 133ms/epoch - 4ms/step
Epoch 36/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0059 - 129ms/epoch - 4ms/step
Epoch 37/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0054 - 124ms/epoch - 4ms/step
Epoch 38/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0055 - 130ms/epoch - 4ms/step
Epoch 39/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0058 - 131ms/epoch - 4ms/step
Epoch 40/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0054 - 126ms/epoch - 4ms/step
Epoch 41/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0056 - 142ms/epoch - 4ms/step
Epoch 42/100
35/3

Epoch 29/100
70/70 - 0s - loss: 0.0319 - val_loss: 0.0397 - 186ms/epoch - 3ms/step
Epoch 30/100
70/70 - 0s - loss: 0.0316 - val_loss: 0.0395 - 198ms/epoch - 3ms/step
Epoch 31/100
70/70 - 0s - loss: 0.0313 - val_loss: 0.0391 - 188ms/epoch - 3ms/step
Epoch 32/100
70/70 - 0s - loss: 0.0311 - val_loss: 0.0387 - 188ms/epoch - 3ms/step
Epoch 33/100
70/70 - 0s - loss: 0.0308 - val_loss: 0.0385 - 191ms/epoch - 3ms/step
Epoch 34/100
70/70 - 0s - loss: 0.0306 - val_loss: 0.0381 - 188ms/epoch - 3ms/step
Epoch 35/100
70/70 - 0s - loss: 0.0304 - val_loss: 0.0379 - 192ms/epoch - 3ms/step
Epoch 36/100
70/70 - 0s - loss: 0.0302 - val_loss: 0.0378 - 183ms/epoch - 3ms/step
Epoch 37/100
70/70 - 0s - loss: 0.0300 - val_loss: 0.0374 - 192ms/epoch - 3ms/step
Epoch 38/100
70/70 - 0s - loss: 0.0299 - val_loss: 0.0373 - 190ms/epoch - 3ms/step
Epoch 39/100
70/70 - 0s - loss: 0.0297 - val_loss: 0.0372 - 191ms/epoch - 3ms/step
Epoch 40/100
70/70 - 0s - loss: 0.0296 - val_loss: 0.0371 - 184ms/epoch - 3ms/step
Epoc

Epoch 28/100
35/35 - 0s - loss: 0.0046 - val_loss: 0.0064 - 135ms/epoch - 4ms/step
Epoch 29/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0062 - 133ms/epoch - 4ms/step
Epoch 30/100
35/35 - 0s - loss: 0.0045 - val_loss: 0.0062 - 132ms/epoch - 4ms/step
Epoch 31/100
35/35 - 0s - loss: 0.0044 - val_loss: 0.0062 - 133ms/epoch - 4ms/step
Epoch 32/100
35/35 - 0s - loss: 0.0044 - val_loss: 0.0062 - 130ms/epoch - 4ms/step
Epoch 33/100
35/35 - 0s - loss: 0.0043 - val_loss: 0.0060 - 143ms/epoch - 4ms/step
Epoch 34/100
35/35 - 0s - loss: 0.0043 - val_loss: 0.0058 - 133ms/epoch - 4ms/step
Epoch 35/100
35/35 - 0s - loss: 0.0042 - val_loss: 0.0058 - 131ms/epoch - 4ms/step
Epoch 36/100
35/35 - 0s - loss: 0.0041 - val_loss: 0.0055 - 134ms/epoch - 4ms/step
Epoch 37/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0055 - 131ms/epoch - 4ms/step
Epoch 38/100
35/35 - 0s - loss: 0.0040 - val_loss: 0.0053 - 127ms/epoch - 4ms/step
Epoch 39/100
35/35 - 0s - loss: 0.0039 - val_loss: 0.0051 - 136ms/epoch - 4ms/step
Epoc

Epoch 27/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0329 - 205ms/epoch - 3ms/step
Epoch 28/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0342 - 205ms/epoch - 3ms/step
Epoch 29/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0337 - 210ms/epoch - 3ms/step
Epoch 30/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0345 - 209ms/epoch - 3ms/step
Epoch 31/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0364 - 198ms/epoch - 3ms/step
Epoch 32/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0338 - 209ms/epoch - 3ms/step
Epoch 33/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0349 - 214ms/epoch - 3ms/step
Epoch 34/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0347 - 203ms/epoch - 3ms/step
Epoch 35/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0339 - 213ms/epoch - 3ms/step
Epoch 00035: early stopping
Epoch 1/100
109/109 - 1s - loss: 0.0540 - val_loss: 0.0541 - 1s/epoch - 12ms/step
Epoch 2/100
109/109 - 0s - loss: 0.0277 - val_loss: 0.0349 - 269ms/epoch - 2ms/step
Epoch 3/100
109/109 - 0s - loss: 0.0093 - val_loss: 0.0262 

Epoch 89/100
109/109 - 0s - loss: 0.0020 - val_loss: 0.0088 - 266ms/epoch - 2ms/step
Epoch 90/100
109/109 - 0s - loss: 0.0020 - val_loss: 0.0088 - 269ms/epoch - 2ms/step
Epoch 91/100
109/109 - 0s - loss: 0.0020 - val_loss: 0.0087 - 271ms/epoch - 2ms/step
Epoch 92/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0087 - 264ms/epoch - 2ms/step
Epoch 93/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0086 - 269ms/epoch - 2ms/step
Epoch 94/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0087 - 274ms/epoch - 3ms/step
Epoch 95/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0087 - 275ms/epoch - 3ms/step
Epoch 96/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0086 - 265ms/epoch - 2ms/step
Epoch 97/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0088 - 220ms/epoch - 2ms/step
Epoch 98/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0086 - 211ms/epoch - 2ms/step
Epoch 99/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0087 - 219ms/epoch - 2ms/step
Epoch 100/100
109/109 - 0s - loss: 0.0019 - val_loss: 0.0086 - 22